In [1]:
import pandas as pd 
import numpy as np 
import plotly.express as px

In [12]:
df = pd.read_csv("reading_data.csv")
df.head()

,Title,Author,Genre,Trope,Date Finished,Rating,Notes,Description - ChatGpt,Month,Year
0,Grayson's Vow,Mia Sheridan,Romance,Fake Relationship,2022-08-20,4,"After reading Archer's Voice, I had set the ba...",Marriage of convenience where the female lead ...,8,2022
1,The Contract,Melanie Moreland,Romance,Fake Relationship,2022-08-30,5,I absolutely loved this book. The author did a...,A ruthless boss and his fed-up assistant find ...,8,2022
2,The Favor,Suzanne Wright,Romance,Fake Relationship,2022-08-30,5,Absolutely loved this book. The first book tha...,Vienna Stratton reluctantly accepted help from...,8,2022
3,Drago,Sarah Brianne,Romance,Mafia,2022-12-14,2,Did not like this book at all. There was poten...,"In a gritty love story, Drago, the Caruso fami...",12,2022
4,Twisted Hate,Ana Huang,Romance,Enemies to Lovers,2022-12-11,4,This book was a good meh. I really liked the c...,"Josh can't stand Jules, but their fiery animos...",12,2022


In [3]:
romance_only = df[df["Genre"] == "Romance"]
counts_romance = romance_only.groupby("Trope").count()["Title"].reset_index()
counts_romance = counts_romance.rename(columns={"Title":"Count"})
counts_romance

,Trope,Count
0,Age Gap,1
1,Bestfriend's Brother,3
2,Children in Tow,7
3,Dark,5
4,Enemies to Lovers,3
5,Fake Relationship,10
6,Heart Breaking,5
7,Mafia,3
8,Misc.,2
9,Second Chance Love,5


In [4]:
def bar_chart(df,x, y, title): 
    fig = px.bar(df, x=x, y=y, title=title)
    fig.update_traces(marker_color='rgba(3, 142, 177, 0.959)')
    fig.show()

In [5]:
# Bar chart for count of tropes
bar_chart(counts_romance, x="Trope", y="Count", title="Breakdown of Count of Tropes Since 2022")

In [6]:
genres_count = df.groupby("Genre")["Title"].count().reset_index()
genres_count = genres_count.rename(columns={"Title":"Count"})
genres_count

,Genre,Count
0,Magical Realism,1
1,Memoir,1
2,Mental Health - Fiction,4
3,Mystery,3
4,Nonfiction,1
5,Psychological Thriller,1
6,Romance,59


In [7]:
bar_chart(genres_count, x="Genre", y="Count", title="Breakdown of Genres Read Since 2022")

In [8]:
year_count = df.groupby("Year")['Title'].count().reset_index()
year_count = year_count.rename(columns={"Title":"Count"})
year_count

,Year,Count
0,2022,12
1,2023,29
2,2024,29


In [11]:
# Create a dash drop down --> selecting the year, shows the number of books read per month
# also shows the average rating of the books 
# THIS IS WHAT I'LL INCLUDE IN THE DASH_BACKEND.PY


year_month_count = df.groupby(["Year", "Month"])["Title"].count().reset_index()
year_month_count = year_month_count.rename(columns={"Title":"Count"})
year_month_count

count_year = df.groupby("Year")["Title"].count().reset_index()
count_year = count_year.rename(columns={"Title":"Count"})

average_year = df.groupby("Year")["Rating"].mean()
year_month_count

,Year,Month,Count
0,2022,3,3
1,2022,8,3
2,2022,11,1
3,2022,12,5
4,2023,1,7
5,2023,2,1
6,2023,4,1
7,2023,5,2
8,2023,7,2
9,2023,8,3


,Year,Month,Count
0,2022,3,3
1,2022,8,3
2,2022,11,1
3,2022,12,5
4,2023,1,7
5,2023,2,1
6,2023,4,1
7,2023,5,2
8,2023,7,2
9,2023,8,3


In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Initialize the Dash app
app = dash.Dash(__name__)

# App layout
app.layout = html.Div([
    # Dropdown for selecting the year
    dcc.Dropdown(
        id='year-dropdown',
        options=[{'label': str(year), 'value': year} for year in sorted(year_month_count['Year'].unique())],
        value=2022,  # Default value
        clearable=False
    ),
    
    # Line chart
    dcc.Graph(id='line-chart')
])

# Callback to update the line chart based on selected year
@app.callback(
    Output('line-chart', 'figure'),
    [Input('year-dropdown', 'value')]
)
def update_line_chart(selected_year):
    # Filter the DataFrame for the selected year
    filtered_df = year_month_count[year_month_count['Year'] == selected_year]
    
    # Create a line chart using Plotly
    fig = px.line(filtered_df, x='Month', y='Count', title=f'Count for Each Month in {selected_year}')
    
    # Update the x-axis to show month labels (1 to 12)
    fig.update_layout(xaxis=dict(tickmode='array', tickvals=list(range(1, 13))))
    
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


In [ ]:
fig = px.scatter(year_month_count, x=['Year', 'Month'], y='Count',size='Count', color='Count')
fig.show()

In [ ]:
fig = px.scatter(df, x='Year', y='Month',size='value', color='value')
